In [ ]:
from h3 import h3
import numpy as np
import cv2
import glob
from matplotlib import pyplot as plt
import os
import pandas as pd
from osgeo import gdal
from osgeo import osr
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
# seaborn stacked bar plot per frame the stationary and moving people
import seaborn as sns
import matplotlib.ticker as mtick
def imshow(image, show_axes = False, quiet = False):
    if len(image.shape) == 3:
      # Height, width, channels
      # Assume BGR, do a conversion since 
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    else:
      # Height, width - must be grayscale
      # convert to RGB, since matplotlib will plot in a weird colormap (instead of black = 0, white = 1)
      image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    # Draw the image
    plt.imshow(image)
    if not show_axes:
        # We'll also disable drawing the axes and tick marks in the plot, since it's actually an image
        plt.axis('off')
    if not quiet:
        # Make sure it outputs
        plt.show()

# Goal: Calculate speed for further analysis

In [ ]:
from tqdm import tqdm
outputfolder = "../../_data/05_tracking_result_projected/step0_attr_prj"
outputfolder_2 = "../../_data/05_tracking_result_projected/step0_no_attr_prj"

# outputfolder = "../../_data/05_tracking_result_projected/step0_attr_prj_speed_05"


def get_frame_num(time_str, fps = 29.97002997002997):
    try:
        time = time_str.split(" ")[0][3:].split(":")
        minute = int(time[0])
        second = int(time[1])
        frame = minute*60*fps + second*fps
        return int(frame)
    except:
        return 0

In [ ]:
# aggregate unique number of people appeared for each frame on a time series
# load a sample video first
def load_video(videoname, outputfolder):

    loc_name = videopath_sel[videopath_sel['video_id']==videoname]['video_location'].values[0]
    video_start_frame = videopath_sel[videopath_sel['video_id']==videoname]['frame_start'].values[0]
    video_start_at = videopath_sel[videopath_sel['video_id']==videoname]['video_section_started_at'].values[0]
    video_group = videopath_sel[videopath_sel['video_id']==videoname]['video_group'].values[0]

    destfolder = os.path.join(outputfolder, loc_name)
    traceGDF = pd.read_csv(os.path.join(destfolder, f"{videoname}_projected.csv"))
    traceGDF = traceGDF[traceGDF['frame_id']>video_start_frame].reset_index(drop = True)
    traceGDF['timestamp'] = video_start_at + traceGDF['frame_id'].apply(lambda x: pd.Timedelta(seconds = x/29.97))
    traceGDF['video_group'] = video_group
    traceGDF['videoname'] = videoname
    
    
    return traceGDF

# drop the outlier speed
def find_outliers_IQR(df, field):

   q1=df[field].quantile(0.25)

   q3=df[field].quantile(0.75)

   IQR=q3-q1

   outliers = df[((df[field]<(q1-1.5*IQR)) | (df[field]>(q3+1.5*IQR)))]

   keep = df[((df[field]>=(q1-1.5*IQR)) & (df[field]<=(q3+1.5*IQR)))]

   return outliers, keep

In [ ]:
# use one video as an example
# consider the fact that same person can stop and move. So we calculate the 5 second average for each person
def get_speed_vector(keepGDF, n = 2, fps = 29.97, globalcrs = "EPSG:3857"):
    """This function calculate the speed vector for each track"""
    keepGDF = gpd.GeoDataFrame(keepGDF, geometry = [Point(x,y) for x,y in zip(keepGDF['lon'], keepGDF['lat'])], crs = f"EPSG:4326")
    keepGDF = keepGDF.to_crs(globalcrs)
    
    # calculate individual walking speed at each frame
    # step 1: calculate distance between every n seconds
    shift_inter = int(n*fps)
    keepGDF = keepGDF.sort_values(['track_id','frame_id', 
                                   ]).reset_index(drop = True)

    
    keepGDF['dist'] = keepGDF.groupby('track_id')['geometry'].transform(lambda x: x.shift(2).distance(x)).fillna(method='bfill')
        
    # break the track_id when the move_m_{n}s is too large
    threshold = 2.5 # maximum walking speed
    distancemax = n*threshold
    keepGDF['track_id_backup'] = keepGDF['track_id']
    keepGDF['track_id_break'] = np.where(keepGDF['dist']>2, 1, 0)
    keepGDF['track_id_update'] = keepGDF['track_id_break'].fillna(0).astype(int)
    keepGDF['track_id_update'] = keepGDF.groupby('track_id')['track_id_update'].cumsum()
    keepGDF['track_id_combo'] = keepGDF['track_id'].astype(int).astype(str) + "_" + keepGDF['track_id_update'].astype(str)
    keepGDF['track_id'] = keepGDF['track_id_combo']
    
    
    # recompute the speed from here:
    keepGDF[f'move_m_{n}s'] = keepGDF.groupby('track_id')['geometry']\
        .transform(lambda x: x.shift(shift_inter).distance(x))
    
    keepGDF[f'speed_{n}s'] = keepGDF[f'move_m_{n}s']/n
    keepGDF[f'speed_{n}s'] = keepGDF.groupby('track_id')[f'speed_{n}s'].fillna(method = 'bfill')
    # calcualte speed_x and speed_y for each person
    keepGDF['x_3857'] = keepGDF['geometry'].x
    keepGDF['y_3857'] = keepGDF['geometry'].y
    keepGDF[f'dist_x_{n}s'] = keepGDF.groupby('track_id')['x_3857'].transform(lambda x: x.shift(shift_inter).fillna(method = 'bfill')-x)
    keepGDF[f'dist_y_{n}s'] = keepGDF.groupby('track_id')['y_3857'].transform(lambda x: x.shift(shift_inter).fillna(method = 'bfill')-x)
    keepGDF[f'speed_x_{n}s'] = keepGDF[f'dist_x_{n}s']/n
    keepGDF[f'speed_y_{n}s'] = keepGDF[f'dist_y_{n}s']/n
    return keepGDF

def select_thred(traceGDF, i, unit = 'frame_id'):
    # traceGDF[unit] = traceGDF['frame_id'].apply(lambda x: int(x/29.97))
    summary = traceGDF.groupby([unit,f'stationary_{i+1}'])['track_id'].nunique().reset_index()\
        .pivot(columns = f'stationary_{i+1}', index = unit, values = 'track_id').reset_index().fillna(0)\
            .rename(columns = {0:'moving', 1:'stationary'})
    summary_2 = traceGDF.groupby([unit])['track_id'].nunique().reset_index().fillna(0)
    summary_all = summary.merge(summary_2, on = unit, how = 'left').rename(columns = {'track_id':'total_pedestrian'})

    # summary = summary.set_index(unit)
    return summary_all

In [ ]:
# stagingfolder = "../../_data/05_tracking_result_projected/step1_speed_vector"
stagingfolder = "../../_data/05_tracking_result_projected/step1_speed_vector_05"
if not os.path.exists(stagingfolder):
    os.mkdir(stagingfolder)

In [ ]:
videopath = pd.read_csv("../../_data/00_raw/_video_meta/video_path_0509.csv")
videopath['video_id'] = videopath['video_name'].apply(lambda x: x.split(".")[0])

# videopath_sel = videopath[videopath['scene'].isin([2,3])]
videopath_sel = videopath[~videopath['ref_path'].isna()].reset_index(drop = True)
videopath_sel['first_effective_time'].unique()
videopath_sel['first_effective_time'] = videopath_sel['first_effective_time'].fillna("12:00:00 AM")
videopath_sel['frame_start'] = videopath_sel['first_effective_time'].apply(lambda x: get_frame_num(x))
videopath_sel['frame_end'] = videopath_sel['last_effective_time'].apply(lambda x: get_frame_num(x))
videopath_sel['frame_end'] = videopath_sel['frame_end'].fillna(videopath_sel['length'])
videopath_sel['ref_epsg'] = videopath_sel['ref_epsg'].astype(int)
videopath_sel['video_date'] = videopath_sel['video_group_started_at'].apply(lambda x: x.split(" ")[0])
videopath_sel['video_section_started_at'] = pd.to_datetime(videopath_sel['video_date']+ " " +videopath_sel['video_section_started_at'])
videols = videopath_sel[videopath_sel['video_location']!='bryant_park']['video_id'].unique().tolist()

In [ ]:
videols = videopath_sel['video_id'].unique().tolist()


In [ ]:
# finished = os.listdir(stagingfolder)
# finished_name = [x.split(".")[0] for x in finished]
# remain = [x for x in videols if x not in finished_name]
# remain = os.listdir(stagingfolder)
remain = os.listdir(outputfolder_2+'/Met Steps videos (NEW)')
remain = [x.split('_')[0] for x in remain]
len(remain)

In [ ]:
n = 0.5
# fullGDF = []
for videoname in tqdm(remain):
    # try:
    traceGDF = load_video(videoname, outputfolder_2) # change here
    
    traceGDF = get_speed_vector(traceGDF, n, fps = 29.97, globalcrs = "EPSG:3857")
    traceGDF['video_location'] = videopath_sel[videopath_sel['video_id']==videoname]['video_location'].values[0]
    traceGDF.drop('geometry', axis = 1).to_csv(os.path.join(stagingfolder, f"{videoname}.csv"), index = False)
        # fullGDF.append(traceGDF)
    # except:
    #     print(videoname, ": failed")


In [ ]:
sample = traceGDF[traceGDF['track_id_backup']==1].reset_index(drop = True)
ax = sample[sample['speed_0.5s']<=2].plot(column = 'track_id', legend = True)
# sample[sample['speed_0.5s']>2].plot(color = 'red', ax = ax)

In [ ]:
sampleupdate = sample.groupby('track_id')['frame_id'].nunique().sort_values().reset_index()
sletrack = sampleupdate[sampleupdate['frame_id']>1]['track_id'].unique().tolist()
sample = sample[sample['track_id'].isin(sletrack)]
ax = sample.plot(column = 'track_id', legend = True)

In [ ]:
# convert to LinString
sampleline = sample.sort_values('frame_id').groupby('track_id').apply(lambda x: LineString(x[['lon', 'lat']].values.tolist()))\
    .reset_index().rename(columns = {0:'geometry'})
sampleline = gpd.GeoDataFrame(sampleline, geometry = 'geometry', crs = "EPSG:4326")
sampleline.head()

In [ ]:
sample.to_file('test_update.geojson', driver='GeoJSON')

In [ ]:
oldstaging = '../../_data/05_tracking_result_projected/step1_speed_vector'
files = os.listdir(oldstaging)
filenames = [x.split(".")[0] for x in files]
finished = os.listdir(stagingfolder)

In [ ]:
# other data beyond the MET
n = 0.5
# fullGDF = []
for videoname in tqdm(filenames):
    try:
        traceGDF = load_video(videoname, outputfolder) # change here
        traceGDF = get_speed_vector(traceGDF, n, fps = 29.97, globalcrs = "EPSG:3857")
        traceGDF['video_location'] = videopath_sel[videopath_sel['video_id']==videoname]['video_location'].values[0]
        traceGDF.drop('geometry', axis = 1).to_csv(os.path.join(stagingfolder, f"{videoname}.csv"), index = False)
        # fullGDF.append(traceGDF)
    except:
        print(videoname, ": failed")

In [ ]:

# stationary_frame = select_thred(0, unit = 'frame_id')
# stationary_frame.head()

# Test visualize on the videos

In [ ]:
videoname = '20081008-141944b01'

traceGDF = pd.read_csv(os.path.join(stagingfolder, f"{videoname}.csv"))

In [ ]:
summary = traceGDF.groupby('track_id_backup')['track_id'].nunique().sort_values().reset_index()
summary[summary['track_id']>1].shape

In [ ]:
sample = traceGDF[traceGDF['track_id_backup']==1069]
# sample

In [ ]:
# load all video paths
videofolder = "/Users/yuan/Dropbox (MIT)/whyte_CV/_data/00_raw/videos_current_highres"
videos = glob.glob(os.path.join(videofolder, "*/*.avi"))
videos2 = glob.glob(os.path.join(videofolder, "*/*/*.avi"))
videos = videos+videos2
# file_path = "../data/00_raw/videos_current_highres/"+videoname+".avi"
# video, fps, size = getbasics(file_path)
video_df = pd.DataFrame({
    "video_name": [os.path.basename(x) for x in videos],
    "video_path": videos,
    "video_id": [os.path.basename(x).split(".")[0] for x in videos],
})
video_df.shape

In [ ]:
# videoname = remain[-1]
file_path = video_df[video_df["video_id"]==videoname]["video_path"].values[0]

In [ ]:
# Visualize by comparing the group activity and single activity
def getbasics(file_path):
    video = cv2.VideoCapture(file_path)
    fps = video.get(cv2.CAP_PROP_FPS)
    print('frames per second =',fps)
    size = (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    print('frames size =',size)
    # video.release()
    return video, fps, size

def compute_color_for_labels(label):
    """
    Simple function that adds fixed color depending on the class
    """
    palette = (2 ** 11 - 1, 2 ** 15 - 1, 2 ** 20 - 1)
    color = [int((p * (label ** 2 - label + 1)) % 255) for p in palette]
    return tuple(color)

In [ ]:
# MET
def load_met_pred(videoname):

    result_folder = "/Users/yuan/Dropbox (MIT)/whyte_CV/_data/03_tracking_result/_current_video_no_attr"
    predpath = os.path.join(result_folder, f'{videoname}.txt')
    trace = pd.read_csv(predpath, sep = '\t', header = None)
    trace.columns = [ "x1", "y1", "x2", "y2", "track_id", "frame_id"]
    trace['w'] = trace['x2'] - trace['x1']
    trace['h'] = trace['y2'] - trace['y1']
    trace['ratio'] = trace['w']/trace['h']
    return trace

def load_pred(videoname):
    result_folder = "../../_data/06_attr_result/" # with attributes
    # result_folder = '/Users/yuan/Dropbox (MIT)/whyte_CV/_data/03_tracking_result/_current_video_no_attr' # without attributes
    # predpath = os.path.join(result_folder, f'{videoname}_attr_mot.csv')
    predpath = os.path.join(result_folder, f'{videoname}.csv')
    trace = pd.read_csv(predpath)
    trace.rename(columns = {"x":"bbox0", "y":"bbox1", "w":"bbox2", "h":"bbox3"}, inplace = True)
    for x in ["bbox0", "bbox1", "bbox2", "bbox3"]:
        trace[x] = trace[x].astype(int)
    # convert bbox to x,y
    # trace['x1'] = trace['bbox0']
    # trace['x2'] = trace['bbox0']+trace['bbox1']
    # trace['y1'] = trace['bbox2']
    # trace['y2'] = trace['bbox2']+trace['bbox3']
    return trace


# trace.rename(columns = {"x1":"bbox0", "y1":"bbox1", "w":"bbox2", "h":"bbox3"}, inplace = True)

# trace['loc_x'] = (trace['bbox0'] + trace['bbox0'] + trace['bbox2'])/2
# trace['loc_y'] = (trace['bbox1'] + trace['bbox3'])

In [ ]:
# trace_sel_merg = trace.merge(sample, left_on = ['track_id', 'frame_id'], 
#                              right_on = ['track_id_backup', 'frame_id'], 
#                              how = 'inner')
# trace_sel_merg

In [ ]:
clipfolder = "../../_data/99_result_sample_export2"
if not os.path.exists(clipfolder):
    os.makedirs(clipfolder)

In [ ]:
file_path = video_df[video_df["video_id"]==videoname]["video_path"].values[0]

In [ ]:
import random
dur = sample['frame_id'].max() - sample['frame_id'].min() # second, change here for other options
# startframe = int(newgroup["frame_id"].min())
startframe = sample['frame_id'].min()
endframe = int(startframe + dur)

# set startframe
video, fps, size = getbasics(file_path)
video.set(cv2.CAP_PROP_POS_FRAMES, startframe)
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
video_out_name = videoname + f"_{startframe}_{endframe}_group"
video_out = cv2.VideoWriter(os.path.join(clipfolder, 
                                         f"{video_out_name}.mp4"), fourcc, fps, size)

In [ ]:
ret = True
count = 0
while ret and count<endframe:
    ret, frame = video.read()
    frame_id = count
    # data = trace_sel_before_pro[trace_sel_before_pro["frame_id"]==frame_id].reset_index(drop = True)
    data = trace_sel_merg[trace_sel_merg['frame_id']==frame_id].reset_index(drop = True)
    # only viz the stationary people
    # data = temp[temp['stationary_1']==1].reset_index(drop = True)
    
    if data.shape[0]>0:
        for i in range(data.shape[0]):
            track_id = data.at[i,'track_id_update']+1
            color = compute_color_for_labels(track_id)
            # assign color to each track

            cv2.rectangle(frame,
                            (data.at[i,'x1'], 
                        data.at[i,'y1']), 
                        (data.at[i,'x2'], 
                        data.at[i,'y2']), 
                    color, 2)

        
    else:
        print("no observation at frame", frame_id)
    if count>=startframe:
        video_out.write(frame) 
    count = count+1
    # print(ret)
video_out.release()

In [ ]:
trace = load_pred(videoname)
trace_sel_merg = trace.merge(sample, left_on = ['track_id', 'frame_id'], 
                             right_on = ['track_id_backup', 'frame_id'], 
                             how = 'inner')
trace_sel_merg.head()

In [ ]:
import random
dur = sample['frame_id'].max() - sample['frame_id'].min() # second, change here for other options
# startframe = int(newgroup["frame_id"].min())
startframe = sample['frame_id'].min()
endframe = int(startframe + dur)

# set startframe
video, fps, size = getbasics(file_path)
video.set(cv2.CAP_PROP_POS_FRAMES, startframe)
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
video_out_name = videoname + f"_{startframe}_{endframe}_group"
video_out = cv2.VideoWriter(os.path.join(clipfolder, 
                                         f"{video_out_name}.mp4"), fourcc, fps, size)

In [ ]:
ret = True
count = startframe
while ret and count<endframe:
    ret, frame = video.read()
    frame_id = count
    # data = trace_sel_before_pro[trace_sel_before_pro["frame_id"]==frame_id].reset_index(drop = True)
    data = trace_sel_merg[trace_sel_merg['frame_id']==frame_id].reset_index(drop = True)
    # only viz the stationary people
    # data = temp[temp['stationary_1']==1].reset_index(drop = True)
    
    if data.shape[0]>0:
        for i in range(data.shape[0]):
            track_id = data.at[i,'track_id_update']+1
            color = compute_color_for_labels(track_id)
            # assign color to each track

            cv2.rectangle(frame,
                            (data.at[i,'bbox0'], 
                        data.at[i,'bbox1']), 
                        (data.at[i,'bbox0']+data.at[i,'bbox2'], 
                        data.at[i,'bbox1']+data.at[i,'bbox3']), 
                    color, 2)

        
    else:
        print("no observation at frame", frame_id)
    if count>=startframe:
        video_out.write(frame) 
    count = count+1
    # print(ret)
video_out.release()

## for stationary detection

In [ ]:
ret = True
count = startframe
frame_total = endframe - startframe

while ret and count<=frame_total+startframe:
    ret, frame = video.read()
    frame_id = count
    temp = vizdata[vizdata["frame_id"]==frame_id].reset_index(drop = True)
    # only viz the stationary people
    data = temp[temp['stationary_3']==1].reset_index(drop = True)
    if data.shape[0]>0:
        for i in range(data.shape[0]):
            track_id = data.at[i,'track_id']
            cv2.rectangle(frame, 
                        (data.at[i,'bbox0'], 
                        data.at[i,'bbox1']), 
                        (data.at[i,'bbox0']+data.at[i,'bbox2'], 
                        data.at[i,'bbox1']+data.at[i,'bbox3']), 
                            (0,0,255), 2)
            cv2.putText(
                    frame, 
                    str(int(track_id)),
                    (data.at[i,'bbox0'], data.at[i,'bbox1']-10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, 
                    2,
                    lineType=cv2.LINE_AA
                )

    # print(frame, ret)
        
    else:
        print("no stationary at frame", frame_id)
    video_out.write(frame) 
    count = count+1
    # print(ret)
video_out.release()